In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
datapath = r"C:\Users\vedan\Downloads\data\data"
outputmodel = r"C:\Users\vedan\Downloads\data\video_classification_model\videoclassificationmodel"
outputlabelbinarizer = r"C:\Users\vedan\Downloads\data\model\videoclassificationoinarizer"
epoch = 25

In [3]:
from sklearn.preprocessing import LabelBinarizer

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
sports_labels = set(["basketball", "ballet", "dancehall", "hiphop", "house", "salsa", "jazz", "indianclassical"])
print("images is being uploaded.....")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in sports_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 244))
    data.append(image)
    labels.append(label)


images is being uploaded.....


In [7]:
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [8]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [10]:
# print(X_train)
print(type(labels))

<class 'numpy.ndarray'>


In [10]:
trainAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 110.779, 103.939], dtype="float32")
trainAugmentation.mean = mean
validationAugmentation.mean = mean

In [11]:
# from keras.applications import ResNet50
from keras.applications import VGG19

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout

from keras.models import Model
from keras.optimizers import SGD

In [12]:
# baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,244,3)))
baseModel = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,244,3)))

In [13]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
# headModel = Reshape((64, -1))(headModel)
# # Just write this Permute after, its complicated why
# headModel = Permute((2, 1))(headModel)
# # it can also be an LSTM
# headModel = SimpleRNN(128, activation='relu')(headModel)
# headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

opt = SGD(lr = 0.0001, momentum=0.9, decay=1e-4/epoch)
# https://keras.io/api/optimizers/sgd/
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 244, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 122, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 122, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 122, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 61, 128)      

In [14]:
opt = SGD(lr = 0.0001, momentum=0.9, decay=1e-4/epoch)
# https://keras.io/api/optimizers/sgd/

In [15]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [16]:
History = model.fit_generator(
    trainAugmentation.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=validationAugmentation.flow(X_test, Y_test),
    validation_steps=len(X_test) // 32,
    epochs=epoch
)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
92/92 [==============================] - 409s 4s/step - loss: 3.8112 - accuracy: 0.3242 - val_loss: 1.2069 - val_accuracy: 0.6219
Epoch 2/25
92/92 [==============================] - 435s 5s/step - loss: 1.7467 - accuracy: 0.5580 - val_loss: 0.8000 - val_accuracy: 0.7250
Epoch 3/25
92/92 [==============================] - 442s 5s/step - loss: 1.3218 - accuracy: 0.6477 - val_loss: 0.5685 - val_accuracy: 0.7896
Epoch 4/25
92/92 [==============================] - 455s 5s/step - loss: 0.9754 - accuracy: 0.7097 - val_loss: 0.4543 - val_accuracy: 0.8354
Epoch 5/25
92/92 [==============================] - 457s 5s/step - loss: 0.8656 - accuracy: 0.7360 - val_loss: 0.4031 - val_accuracy: 0.8646
Epoch 6/25
92/92 [==============================] - 443s 5s/step - loss: 0.7106 - accuracy: 0.7802 - val_loss: 0.3615 - val_accuracy: 0.8760
Epoch 7/25
92/92 [==============================] - 3218s 35s/step - loss: 0.6

In [5]:
import pickle
outputmodel = r"C:\Users\vedan\Downloads\data\video_classification_model\videoclassificationmodel"


In [18]:
model.save(outputmodel)
lbinarizer = open("data_VGG19.pickle", "wb")
with open('data_.pickle', 'wb') as f:
        pickle.dump(lb, f)
# lbinarizer.write(pickle.dump(lb))
lbinarizer.close()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\vedan\Downloads\data\video_classification_model\videoclassificationmodel\assets


In [3]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

In [6]:
model = load_model(outputmodel)
with open('data_.pickle', 'rb') as f:
    lb = pickle.load(f)
# mean = np.array([123.68, 110.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)

In [23]:
outputvideo = "demo_output.avi"

In [25]:
Queue = deque(maxlen=128)

In [26]:
capture_video = cv2.VideoCapture("test_video.mp4")
writer = None
(Width, Height) = (None, None)

while True:
    (taken, frame) = capture_video.read()
    if not taken:
        break
    if Width is None and Height is None:
        (Width, Height) = frame.shape[:2]
    
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 244)).astype("float32")
#     print(frame.shape)
#     frame = mean
    print(frame.shape)
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    text = "They are playing : {}".format(label)
    cv2.putText(output, text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255,0,0), 5)
    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter("outputvideo", fourcc, 30, (Width, Height), True)
    cv2.imshow("In progress" ,output)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break

print("Finalizing.....")
writer.release()
capture_video.release()


(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 224, 3)
(244, 

In [7]:
datapath = r"C:\Users\vedan\Downloads\data\data\test_data"

sports_labels = set(["basketball", "ballet", "dancehall", "hiphop", "house", "salsa", "jazz", "indianclassical"])
print("images is being uploaded.....")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in sports_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 244))
    data.append(image)
    labels.append(label)

images is being uploaded.....


In [10]:
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [11]:
print(data.shape)

(97, 244, 224, 3)


In [15]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.75, stratify=labels, random_state=42)

In [16]:
print(X_test.shape)

(73, 244, 224, 3)


In [17]:
print(len(X_test))
print(len(Y_test))

73
73


In [18]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test)
print("test loss, test acc:", results)

Evaluate on test data
3/3 [==============================] - 5s 2s/step - loss: 0.4431 - accuracy: 0.8630
test loss, test acc: [0.4430887997150421, 0.8630136847496033]


In [8]:
# capture_video = cv2.VideoCapture("demo")

# cv2.imshow("In progress", capture_video)
# key = cv2.waitkey(1) & oxFF
    
# if key == ord("q"):
#     break


In [69]:
# import cv2
# import numpy as np

# # Create a VideoCapture object and read from input file
# # If the input is the camera, pass 0 instead of the video file name
# cap = cv2.VideoCapture('demo.mp4')

# (taken, frame) = cap.read()
# print(frame.shape)
# # Check if camera opened successfully
# if (cap.isOpened()== False): 
#   print("Error opening video stream or file")

# # Read until video is completed
# while(cap.isOpened()):
#   # Capture frame-by-frame
#   ret, frame = cap.read()
#   if ret == True:

#     # Display the resulting frame
#     cv2.imshow('Frame',frame)

#     # Press Q on keyboard to  exit
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#       break

#   # Break the loop
#   else: 
#     break

# # When everything done, release the video capture object
# cap.release()

# # Closes all the frames
# cv2.destroyAllWindows()


(720, 1280, 3)
